In [2]:
import utility.stock

data = utility.stock.get_stock_history('600036','1y')

[*********************100%***********************]  1 of 1 completed


In [3]:
print(data)

                 Open       High        Low      Close  Adj Close     Volume
Date                                                                        
2022-09-08  33.369999  33.529999  33.250000  33.299999  31.556765   41667222
2022-09-09  33.450001  34.759998  33.380001  34.500000  32.693947  135166615
2022-09-13  34.450001  35.650002  34.290001  35.349998  33.499447  143861057
2022-09-14  34.900002  35.299999  34.709999  34.860001  33.035099   56078636
2022-09-15  35.150002  36.060001  35.150002  36.009998  34.124897  134095049
...               ...        ...        ...        ...        ...        ...
2023-09-04  32.580002  33.610001  32.470001  33.529999  33.529999  110429932
2023-09-05  33.299999  33.439999  32.970001  33.110001  33.110001   44732261
2023-09-06  33.099998  33.459999  32.770000  33.360001  33.360001   54674047
2023-09-07  33.310001  33.310001  33.009998  33.139999  33.139999   35870198
2023-09-08  33.000000  33.259998  32.950001  33.200001  33.200001   19961396

In [146]:
import pandas as pd

fast = 12
slow = 26
ma = 9

macd = pd.DataFrame()
macd['close'] = data['Close']
macd = macd.reset_index()
macd = macd.rename(columns={"Date": "date"})

if (slow <= fast):
    print("Error: Slow smaller than Fast")

if len(macd.index) <= slow:
    print("Error: Data size too small")

macd['fast_ma'] = pd.NA
macd['slow_ma'] = pd.NA
macd['dif'] = pd.NA
macd['dea'] = pd.NA
macd['macd'] = pd.NA

macd.loc[fast-1,'fast_ma'] = macd['close'].iloc[0:fast].mean()

# calculate fast_ma from fast and slow
for i in range(fast, slow):
    macd.loc[i,'fast_ma'] = (macd.loc[i,'close'] * (2/(fast+1))) + (macd.loc[i-1,'fast_ma'] * ((fast -1)/(fast+1)))

# calculate fast_ma, slow_ma, dif
macd.loc[slow-1,'slow_ma'] = macd['close'].iloc[0:slow].mean()
macd.loc[slow-1,'dif'] = macd.loc[slow-1, 'fast_ma']  - macd.loc[slow-1, 'slow_ma']

for i in range(slow, len(macd)):
    macd.loc[i,'slow_ma'] = (macd.loc[i,'close'] * (2/(slow+1))) + (macd.loc[i-1,'slow_ma'] * ((slow -1)/(slow+1)))
    macd.loc[i,'fast_ma'] = (macd.loc[i,'close'] * (2/(fast+1))) + (macd.loc[i-1,'fast_ma'] * ((fast -1)/(fast+1)))
    macd.loc[i,'dif'] = macd.loc[i,'fast_ma'] - macd.loc[i,'slow_ma'] 

# calculate dea, macd
macd.loc[slow+ma-2,'dea'] = macd['dif'].iloc[slow-1:slow+ma-1].mean()
macd.loc[slow+ma-2,'macd'] = 2*(macd.loc[slow+ma-2,'dif'] - macd.loc[slow+ma-2,'dea'])

for i in range(slow+ma-1, len(macd)):
    macd.loc[i,'dea'] = (macd.loc[i,'dif'] * (2/(ma+1))) + (macd.loc[i-1,'dea'] * ((ma -1)/(ma+1)))
    macd.loc[i,'macd'] = 2*(macd.loc[i,'dif'] - macd.loc[i,'dea'])

macd = macd.set_index('date')
macd = macd.drop(columns=['close', 'slow_ma', 'fast_ma'])
macd = macd.dropna()

print(macd)
macd.to_csv('test1.csv')


                 dif       dea      macd
date                                    
2022-11-02 -1.890521 -1.988096   0.19515
2022-11-03 -1.802968  -1.95107  0.296205
2022-11-04  -1.63166 -1.887188  0.511056
2022-11-07 -1.472469 -1.804244  0.663551
2022-11-08 -1.331763 -1.709748   0.75597
...              ...       ...       ...
2023-09-04 -0.235291 -0.376669  0.282756
2023-09-05 -0.143848 -0.330105  0.372514
2023-09-06 -0.050622 -0.274208  0.447173
2023-09-07  0.005446 -0.218277  0.447446
2023-09-08  0.054097 -0.163802  0.435799

[211 rows x 3 columns]


In [128]:
import pandas as pd

fast = 12
slow = 26
ma = 9

macd = pd.DataFrame()
macd['close'] = data['Close']
macd = macd.reset_index()
macd = macd.rename(columns={"Date": "date"})

if (slow <= fast):
    print("Error: Slow smaller than Fast")

if len(macd.index) <= slow:
    print("Error: Data size too small")

macd['fast_ma'] = pd.NA
macd['slow_ma'] = pd.NA
macd['dif'] = pd.NA
macd['dea'] = pd.NA
macd['macd'] = pd.NA

macd.loc[0,'fast_ma'] = macd.loc[0,'close']
macd.loc[0,'slow_ma'] = macd.loc[0,'close']

for i in range(1, len(macd)):
    macd.loc[i,'fast_ma'] = (macd.loc[i,'close'] * (2/(fast+1))) + (macd.loc[i-1,'fast_ma'] * ((fast -1)/(fast+1)))
    macd.loc[i,'slow_ma'] = (macd.loc[i,'close'] * (2/(slow+1))) + (macd.loc[i-1,'slow_ma'] * ((slow -1)/(slow+1)))

macd['dif'] = macd['fast_ma'] - macd['slow_ma']
macd['dea'] = macd['dif'].rolling(ma).mean()
macd['macd'] = 2*(macd['dif'] - macd['dea'])

macd = macd.set_index('date')
macd = macd.drop(columns=['close', 'slow_ma', 'fast_ma'])

print(macd)
macd.to_csv('test.csv')

                 dif       dea      macd
date                                    
2022-09-08       0.0       NaN       NaN
2022-09-09  0.095727       NaN       NaN
2022-09-13  0.237441       NaN       NaN
2022-09-14  0.306677       NaN       NaN
2022-09-15  0.449165       NaN       NaN
...              ...       ...       ...
2023-09-04 -0.235291 -0.470881   0.47118
2023-09-05 -0.143848 -0.426523  0.565351
2023-09-06 -0.050622 -0.366523  0.631803
2023-09-07  0.005446 -0.301333  0.613558
2023-09-08  0.054097 -0.238493   0.58518

[244 rows x 3 columns]


In [126]:
import pandas as pd

fast = 12
slow = 26
ma = 9

macd = pd.DataFrame()
macd['close'] = data['Close']
macd = macd.reset_index()
macd = macd.rename(columns={"Date": "date"})

if (slow <= fast):
    print("Error: Slow smaller than Fast")

if len(macd.index) <= slow:
    print("Error: Data size too small")

macd['fast_ma'] = macd['close'].rolling(fast).mean()
macd['slow_ma'] = macd['close'].rolling(slow).mean()
macd['dif'] = macd['fast_ma'] - macd['slow_ma']
macd['dea'] = macd['dif'].rolling(ma).mean()
macd['macd'] = 2*(macd['dif'] - macd['dea'])

macd = macd.set_index('date')
#macd = macd.drop(columns=['close', 'slow_ma', 'fast_ma'])

print(macd)
macd.to_csv('test.csv')

                close    fast_ma    slow_ma       dif       dea      macd
date                                                                     
2022-09-08  33.299999        NaN        NaN       NaN       NaN       NaN
2022-09-09  34.500000        NaN        NaN       NaN       NaN       NaN
2022-09-13  35.349998        NaN        NaN       NaN       NaN       NaN
2022-09-14  34.860001        NaN        NaN       NaN       NaN       NaN
2022-09-15  36.009998        NaN        NaN       NaN       NaN       NaN
...               ...        ...        ...       ...       ...       ...
2023-09-04  33.529999  31.759166  32.838077 -1.078911 -1.140869  0.123917
2023-09-05  33.110001  31.861666  32.743846 -0.882180 -1.168590  0.572820
2023-09-06  33.360001  32.047500  32.681154 -0.633654 -1.142308  1.017308
2023-09-07  33.139999  32.199166  32.646923 -0.447757 -1.073148  1.250784
2023-09-08  33.200001  32.395000  32.590000 -0.195000 -0.956532  1.523063

[244 rows x 6 columns]


In [147]:
import pandas as pd

fast = 12
slow = 26
ma = 9

macd = pd.DataFrame()
macd['close'] = data['Close']
macd = macd.reset_index()
macd = macd.rename(columns={"Date": "date"})

if (slow <= fast):
    print("Error: Slow smaller than Fast")

if len(macd.index) <= slow:
    print("Error: Data size too small")

macd.loc[0,'fast_ma'] = macd.loc[0,'close']
macd.loc[0,'slow_ma'] = macd.loc[0,'close']

macd['fast_ma'] = macd['close'].ewm(alpha=2/(fast+1), adjust=False).mean()
macd['slow_ma'] = macd['close'].ewm(alpha=2/(slow+1), adjust=False).mean()
macd['dif'] = macd['fast_ma'] - macd['slow_ma']
macd['dea'] = macd['dif'].ewm(alpha=2/(ma+1), adjust=False).mean()
macd['macd'] = 2*(macd['dif'] - macd['dea'])

macd = macd.set_index('date')
macd = macd.drop(columns=['close', 'slow_ma', 'fast_ma'])

print(macd)
macd.to_csv('test.csv')

                 dif       dea      macd
date                                    
2022-09-08  0.000000  0.000000  0.000000
2022-09-09  0.095727  0.019145  0.153162
2022-09-13  0.237441  0.062804  0.349273
2022-09-14  0.306677  0.111579  0.390196
2022-09-15  0.449165  0.179096  0.540137
...              ...       ...       ...
2023-09-04 -0.235291 -0.376669  0.282756
2023-09-05 -0.143848 -0.330104  0.372514
2023-09-06 -0.050622 -0.274208  0.447173
2023-09-07  0.005446 -0.218277  0.447445
2023-09-08  0.054097 -0.163802  0.435799

[244 rows x 3 columns]
